In [2]:
import opendatasets as od
#his cell downloads the data needed for this jupyter notebook from kaggle and stores in the-movies-dataset folder
#if the files are already in that folder than this cell does nothing and requires no credentials

#Data Soruce Information:
#https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv

od.download("https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset")

Skipping, found downloaded files in ".\the-movies-dataset" (use force=True to force download)


In [1]:
import pandas as pd
import random


#This cell is for combining certain data from the necessary csv files into a single dataframe (complete)

pd.set_option('display.max_colwidth', None)


movies_full = pd.read_csv('./the-movies-dataset/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
movies_full = movies_full.dropna()
movies_full = movies_full.reset_index()


#filter rows of empty data from movies_full on the columns: "genres", "production_companies"
drop_indices = []
for i in range(len(movies_full)):
    len_1 = len(movies_full.iloc[i].loc["genres"])                   
    if(movies_full.iloc[i].loc["genres"][len_1 -2:] == "[]"):
        drop_indices.append(i)
        continue
    len_2 = len(movies_full.iloc[i].loc["production_companies"])
    if(movies_full.iloc[i].loc["production_companies"][len_2 -2:] == "[]"):
        drop_indices.append(i)    
        continue   

movies_full = movies_full.drop(labels=drop_indices, axis = 0)
movies_full = movies_full.reset_index(names = "index_1")


ratings = pd.read_csv('./the-movies-dataset/ratings.csv', usecols = ("userId", "movieId", "rating"), dtype={"userId": "string","movieId": "string","rating": "string"})
ratings = ratings.rename(columns={"movieId": "id"})
ratings.dropna()
ratings = ratings.reset_index(names = "index_2")


keywords = pd.read_csv('./the-movies-dataset/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})
keywords.dropna()
keywords = keywords.reset_index()

#filter rows of empty data from keyword on the keywords column
drop_indices = []
for i in range(len(keywords)):
    len_1 = len(keywords.iloc[i].loc["keywords"])                   
    if(keywords.iloc[i].loc["keywords"][len_1 -2:] == "[]"):
        drop_indices.append(i)

keywords = keywords.drop(labels=drop_indices, axis = 0)
keywords = keywords.reset_index(names = "index_3")


credits = pd.read_csv("./the-movies-dataset/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})
credits.dropna()
credits = credits.reset_index()

#filter rows of empty data from credits on the cast column 
drop_indices = []
for i in range(len(credits)):
    len_1 = len(credits.iloc[i].loc["cast"])                   
    if(credits.iloc[i].loc["cast"][len_1 -2:] == "[]"):
        drop_indices.append(i)

credits = credits.drop(labels=drop_indices, axis = 0)
credits = credits.reset_index(names = "index_4")


#default merge is inner: this only keeps movies that have the id existing in both dataframes...
complete =  pd.merge(movies_full, ratings, on ="id")
complete =  pd.merge(complete,keywords, on ="id")
complete  = pd.merge(complete,credits, on ="id")


complete = complete.sort_values(by = 'userId')


#use only certain types of columns
complete  = complete.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]

#for testing
print(complete.head())


        userId    id rating               title  \
6566765      1  1246    5.0        Rocky Balboa   
6880303      1  2959    4.0      License to Wed   
2083077      1  2762    4.5  Young and Innocent   
1492304      1  1968    4.0       Fools Rush In   
2638962      1   147    4.5       The 400 Blows   

                                                                                                genres  \
6566765                                                                  [{'id': 18, 'name': 'Drama'}]   
6880303                                                                 [{'id': 35, 'name': 'Comedy'}]   
2083077                                     [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}]   
1492304  [{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]   
2638962                                                                  [{'id': 18, 'name': 'Drama'}]   

                                                        

In [2]:
import ast
import random
import time

start_time = time.time()

# seed for consistent results across runtime
seed_int = 2
random.seed(seed_int)


def condition(array):
    """"
    originally used to filter out the rows of data with empty entries for certain columns
    a method simlair to this is used in the previous cell above to reduce the number of checks
    """
    length = len(array[4])
    if(array[4][length-2:] == "[]"):
        return False
    length = len(array[5])
    if(array[5][length-2:] == "[]"):
        return False
    length = len(array[6])
    if(array[6][length-2:] == "[]"):
        return False
    length = len(array[7])
    if(array[7][length-2:] == "[]"):
        return False   
    #this is this unecessary with the dropNa function in the previous cell:
    # length = len(array[8])
    # if(array[8][length-4:]=="<NA>"):
    #     return False
    # length = len(array[9])
    # if(array[9][length-4:]=="<NA>"):
    #     return False 
    return True


def populate_names(item):
    """used to extract names from the syntax of certain data entries"""
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            temp_dict = ast.literal_eval(item)
            names+=str(temp_dict["name"])
        else:
            temp_dict = ast.literal_eval(item+"}")
            names+=str(str(temp_dict["name"])+" ")
        cnt += 1
    return names


def provide_data(array):
    """extract data from row of complete_array"""
    movie_data = []
    movie_data.append(int(array[0]))
    movie_data.append(int(array[1]))
    movie_data.append(float(array[2]))
    movie_data.append(array[3])  

    movie_data.append(populate_names(array[4]))
    movie_data.append(populate_names(array[5]))
    movie_data.append(populate_names(array[6]))
    movie_data.append(populate_names(array[7]))

    movie_data.append(str(array[8]))
    movie_data.append(str(array[9]))
    return movie_data
    

#LOOK: 
#https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
#iterating over pandas objects is slow!!!
#should check for other datastructures below


#LOOK:
#there is an issue with this!!!
#complete.loc[complete['userId'] == user_id] is too slow since it observes the entire dataframe
#should try to make use of the fact that rows are in user order
#there can be an initial list that is the copy of the dataframe...
#another idea...
#need to populate gaps and list_of_user_ids...



list_of_user_ids = list(complete["userId"].unique())
complete_list = complete.values.tolist()

print("Complete number of users:", len(list_of_user_ids)) #260788

#the complete list of user rows without ratings of the same movie more than once
complete_list_no_dups = []
#distinquish the users the row belongs to 
last_id = -1
#the set of movies that a user has rated
#used to prevent later ratings of a movie that the user has already rated
movie_set = set()
#how many rows a single user takes up for each user in the order of their occurance
gaps = []
#added to gaps when all of a users rows have been counted
gap_len = 0
#prevent adding gap_len to gaps until a real user has been iterated over
first_it = True

#populates gaps and complete_list_no_dups by omitting movies that already have a rating in respect to each user
for row in complete_list:
    if last_id != row[0]:
        movie_set= set()
        complete_list_no_dups.append(row)
        movie_set.add(row[1])
        if not first_it:    
            gaps.append(gap_len)
        gap_len = 1
        first_it = False
    else:
        if row[1] not in movie_set:
            complete_list_no_dups.append(row)
            gap_len+=1
            movie_set.add(row[1])
    last_id = row[0]

#add the last gap_len
gaps.append(gap_len)

user_index = 0
full_index = 0

nof_train_users = 0
nof_test_users =0
train_or_test = []

#Reduce gaps, list_of_user_ids, and complete_list_no_dups to force users to to have a certain range of ratings...
#to be a train or test user
#also populates nof_train_users, nof_test_users, and train_or_test
for _ in range(len(gaps)):
    if 50 <= gaps[user_index] and 70 >= gaps[user_index]:
        full_index+=gaps[user_index]
        user_index+=1
        train_or_test.append(1)
        nof_train_users+=1
    elif 5 <= gaps[user_index] and 10 >= gaps[user_index]:
        full_index+=gaps[user_index]
        user_index+=1
        train_or_test.append(0)
        nof_test_users+=1
    else:
        temp = gaps[user_index]
        del gaps[user_index]
        del list_of_user_ids[user_index]
        del complete_list_no_dups[full_index:full_index+temp]

print("Number of train users before random selection:", nof_train_users) #14314
print("Number of test users before random selection:", nof_test_users) #68048

#LOOK: does the below operation really need to run for the entire data set???
#instead the above operation can include the random selection
#answer: this does not improve processing speed because it will make the the else portion run more
#which is an expensive task. Note: each delete is O(n)

#LOOK: What about filtering users randomly before selecting train and test users??? 
#the problem with this is that it is unknown how many users will be produced of each user type and if it is enough
#the users need to be seperated into train and test users to find out which would 


#These are the average number of ratings to select from each user type
#note since the target is 5000 train users and 1000 test users the extra 200 users leaves room for error to avoid...
#passing less than 5000 or 1000 users
#this would be a problem since cell 5 needs to select (5000 and 1000) users of each type from the pool
avg_nof_train_users_to_select = 5200
avg_nof_test_users_to_select = 1200

#this is a list for all selected users to rows of transformed data for each movie they rated
user_to_data = []

#index of the current movie row 
index  = 0

#this is collected for insight
avg = 0.0
cnt = 0.0

#populate user_to_data from complete_array
for i in range(0, len(list_of_user_ids)):
    #generate a random float to determine a pass for the user
    if(((train_or_test[i]) and (random.random()<float(avg_nof_train_users_to_select/nof_train_users)))
       or ((not train_or_test[i]) and (random.random()<float(avg_nof_test_users_to_select/nof_test_users)))):
        user_to_data.append([])
        last_index = len(user_to_data) -1
        for j in range(index, len(complete_list_no_dups)):
            if complete_list_no_dups[j][0] == list_of_user_ids[i]:
                #orginally: the condition function checked if the movie row had missing values for certain columns and...
                #omitted the movie if it had missing values   
                # if(condition(complete_array[j])):
                    #transform data...
                #a more efficient method is used instead in the second cell  
                transformed = provide_data(complete_list_no_dups[j])
                user_to_data[last_index].append(transformed)    
            else:
                avg += len(user_to_data[last_index])
                cnt+=1
                index = j
                break           
    else:
        index += gaps[i]



#Go through user_to_data and re-index the users in list order since certain users were omitted
#this is for simplicity, readability, and usage in cell 5
for i in range(len(user_to_data)):
    for j in range(len(user_to_data[i])):
        user_to_data[i][j][0] = i


#How many users are in the final user_to_data 
print("Number of users chosen:", len(user_to_data))

#average number of ratings per users
#note: omits the very last user but this makes little difference
print("Average number of ratings for the users chosen:", float(avg/cnt))


print("Minutes taken: ", float((time.time()-start_time)/60))


# recent run:
# time to complete: 12 (minutes)
# Complete number of users: 260788
# Number of train users before random selection: 14314
# Number of test users before random selection: 68048
# Number of users chosen: 6390
# Average number of ratings for the users chosen: 49.07151799687011
# Minutes taken:  12.13312626282374


Complete number of users: 260788
Number of train users before random selection: 14314
Number of test users before random selection: 68048
Number of users chosen: 6390
Average number of ratings for the users chosen: 49.07151799687011
Minutes taken:  12.13312626282374


In [3]:
#save in a constructed_data/constructed_data.csv file so that cells below can run without running this cell and above

import csv
import os

current_directory = os.getcwd()
final_directory = os.path.join(current_directory, 'constructed_data')
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

with open("constructed_data/constructed_data.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    for i in range(len(user_to_data)):
        writer.writerows(user_to_data[i])

In [14]:
#this is a starting point if the data is already saved to the constructed_data/constructed_data.csv
import csv

data_list =[]

with open("constructed_data/constructed_data.csv", 'r', encoding="utf-8") as read_obj:
    csv_reader = csv.reader(read_obj)
    data_list = list(csv_reader)

data_list = data_list[1:]


In [10]:
import random

# seed for consistent results across runtime
seed_int = 2
random.seed(seed_int)


user_to_data_train = []
user_to_data_test = []

first_iteration = True
user_id = -1
ratings = []
for row in data_list:
    if (row[0]!=user_id):
        if(not first_iteration):
            if(5 <= len(ratings) and 10 >= len(ratings)):
                user_to_data_test.append(ratings)
            else:
                user_to_data_train.append(ratings)
        else:
            first_iteration = False
        user_id = row[0]
        ratings = [row]
    else:
        ratings.append(row)

if(5 <= len(ratings) and 10 >= len(ratings)):
    user_to_data_test.append(ratings)
else:
    user_to_data_train.append(ratings)



user_to_data_train = random.sample(user_to_data_train, 5000)
user_to_data_test = random.sample(user_to_data_test, 1000)




[['584', '1717', '2.5', "All the King's Men", 'Drama Thriller', 'Columbia Pictures Corporation Rising Star Relativity Media Phoenix Pictures VIP 4 Medienfonds', 'corruption journalist based on novel blackmail manipulation bodyguard louisiana scandal power governor politics tragedy mistress aristocrat', 'Sean Penn Jude Law Kate Winslet James Gandolfini Patricia Clarkson Anthony Hopkins Kathy Baker Jackie Earle Haley Travis Champagne Connor Fox Montgomery John Mark Ruffalo', 'Some people will do anything to gain power. Some will do anything to keep it.', "The story of an idealist's rise to power in the world of Louisiana politics and the corruption that leads to his ultimate downfall. Based on the1946 Pulitzer Prize-winning novel written by Robert Penn Warren."], ['584', '5902', '3.5', 'A Bridge Too Far', 'Drama History War', 'United Artists Joseph E. Levine Productions', 'netherlands world war ii panzer british soldier', "Dirk Bogarde James Caan Michael Caine Sean Connery Edward Fox Ell

In [11]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
import random
from ordered_set import OrderedSet
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel

# seed for consistent results across runtime
seed_int = 2
random.seed(seed_int)


class user_type_vars():
    """Each of the variables in this class represent all the data structures for a user type"""
    def __init__(self):
        #for each user, a dictionary of movie_id to the movies rating for each movie the user watched
        self.user_to_movie_id_to_rating = [] 

        #for each user, a random choice of movie_id from all the movies the user watched to represent the target movie
        self.user_to_target_movie_id = [] 

        #for each user, this is the index of the users target movie in the order of train movies_in_order
        #(train_users only)
        self.user_to_target_index_full = [] 

        #for each user, includes ratings for all the movies in the entire train set 
        #missing ratings and target movie ratings are set to that movies average rating
        #(train_users only)
        self.user_to_ratings_full = [] 

        #for each user, includes ratings for all the movies in the entire train set
        #the movies mean rating is subtracted from each rating
        #missing ratings and target movie ratings are set to zero
        #(train_users only)
        self.user_to_ratings_full_transform = []

        #for every movie watched by the user_type, a list of ratings
        self.movie_id_to_ratings = dict()

        #this is a set of every unique target movie for the user_type
        self.target_movies = set()

        #all the movies ids in order of the ratings from top to bottom for either the set of train users or the set of test users
        self.movies_in_order = OrderedSet()

        #model input features x
        self.feature_1 = []
        self.feature_2 = []
        self.feature_3 = []

        #model output feature y
        self.user_to_target_rating  = [] 


#user_type_vars can represesent a group of train users and a group of test users
train_users = user_type_vars()
test_users = user_type_vars()


wnl = WordNetLemmatizer()



def load_feature_1_and_2(target_movies, movies_in_order, user_to_data, movie_id_to_ratings, user_to_movie_id_to_rating, user_to_target_movie_id, user_to_target_rating, feature_1, feature_2):
    """
    This is ran once to be used to populate features 1 and 2 for the train_users...
    and ran again to be used to populate features 1 and 2 for the test_users
    It also populates the train and test version of these variables: target_movies, movies_in_order, movie_id_to_ratings,
    user_to_movie_id_to_rating, user_to_target_movie_id, user_to_target_rating
    These variables are used in the load_feature_3 function 
    """ 
    #these are used to calculate the overall train rating
    #the overall_average_train which is overall_rating_sum/overall_rating_count is only set to the output of the "train" function call
    #this is used to fill in ratings for movies that are only target movies for a certain set of users (the rating doesn't exist)
    overall_rating_sum = 0
    overall_rating_count = 0

    for i in range(len(user_to_data)):
        movie_id_to_words = dict()
        movie_id_to_rating = dict()
        cnt = 0
        total =0
        rand_int = random.randint(0, len(user_to_data[i])-1)
        for movie_data in user_to_data[i]:
            if cnt == rand_int:    
                target_movies.add(movie_data[1])
                user_to_target_movie_id.append(movie_data[1])
            else:
                overall_rating_sum += float(movie_data[2])
                overall_rating_count += 1
                total += float(movie_data[2])

                #this only runs when the movie is not the target movie because
                #the target movies are thought to be the movies whose rating is to be predicted...
                #not ratings that are already on record
                if movie_data[1] in movie_id_to_ratings.keys():
                    movie_id_to_ratings[movie_data[1]].append(float(movie_data[2]))
                else:
                    movie_id_to_ratings[movie_data[1]] = [float(movie_data[2])]

            movie_string = ""

            #use this to apply all the text data and combine in to a single list of words (repeats allowed):
            # for index in range (3,len(movie_data)):
            #     if(index!= len(movie_data)-1):
            #         movie_string+= movie_data[index]+" "
            #     else:
            #         movie_string+= movie_data[index]


            #all of the text columns and a few combinations of certain text columns were tested but they were not helpful in...
            #increasing model performance (see README.md)


            #Use this truncated code to only include the genre column strings:
            movie_string = movie_data[4]

            #lematization and conversion to lists of words
            cleaned = remove_stopwords(movie_string)
            cleaned = [wnl.lemmatize(word) for word in cleaned.split(" ")]
            cleaned = [word[:-1] for word in cleaned if word.endswith(".")] + [word for word in cleaned if not word.endswith(".")]

            movie_id_to_words[movie_data[1]] = cleaned
            movie_id_to_rating[movie_data[1]] = float(movie_data[2])
            movies_in_order.add(movie_data[1])
            cnt+=1

        user_to_movie_id_to_rating.append(movie_id_to_rating)

        #the current users set of words from all the movies they rated
        users_words_in_order = OrderedSet()
        for movie_id in movie_id_to_words.keys():
            for word in movie_id_to_words[movie_id]:
                users_words_in_order.add(word)


        word_counts = [] #list of word counts for the users_words_in_order for each movie (excluding target)
        target_word_counts = [] #word counts for the users_words_in_order for the target movie


        #for each movie the user watched record the wordcount for each word in users_words_in_order
        for movie_id in movie_id_to_words.keys():
            if movie_id != user_to_target_movie_id[-1]:
                temp_dict = Counter(movie_id_to_words[movie_id])
                temp_list = []
                for word in users_words_in_order:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                    else:
                        temp_list.append(0)  
                word_counts.append(temp_list)  
            else:
                temp_dict = Counter(movie_id_to_words[movie_id])
                temp_list = []
                for word in users_words_in_order:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])             
                    else:
                        temp_list.append(0) 
                target_word_counts = temp_list
        

        complete_word_counts = word_counts.copy()
        complete_word_counts.append(target_word_counts)
        transformed_word_counts = TfidfTransformer().fit_transform(complete_word_counts).toarray()


        #populate ratings with the exception of the target rating 
        #also record the users target movie rating 
        ratings = []
        for movie_id in movie_id_to_rating.keys():
            if movie_id != user_to_target_movie_id[-1]:
                ratings.append(movie_id_to_rating[movie_id])
            else:
                #this signifies the ratings to be predicted by the model
                user_to_target_rating.append(movie_id_to_rating[movie_id])
    

        def predict():
            "Use the word counts and ratings to add prediction to feature_1 list and feature_2 list"
            #item_1 is unweighted average of all of the users movie
            item_1 = 0 
            #item_2 is weighted average of all the users movies (weights are based on cossine similarity)
            #unless denominator is zero (see below)
            item_2 = 0

            sum = 0
            for i in range(len(ratings)):
                sum += ratings[i]
            item_1 = float(sum/len(ratings))

            cosine_sim = linear_kernel(X = transformed_word_counts[0:-1],Y = [transformed_word_counts[-1]])
            cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))
            numerator = 0
            denominator = 0
            item_2 = item_1
            for i in range(len(ratings)):
                numerator += float(cosine_sim[i]*ratings[i])
                denominator += cosine_sim[i]
    
            #in case of potential division by zero
            if denominator != 0:
                item_2 = float(numerator/denominator)
    
            return (item_1, item_2)
        
        items = predict()

        feature_1.append(items[0])
        feature_2.append(items[1])
            
        
    return float(overall_rating_sum/overall_rating_count)

#populate train data (feature 1 and feature 2)
#the overall_average_train which is overall_rating_sum/overall_rating_count is only set to the output of the "train" function call
#this is used to fill in ratings for movies that are only target movies for a certain set of users (the rating doesn't exist)
overall_average_train = load_feature_1_and_2(train_users.target_movies, train_users.movies_in_order, user_to_data_train, train_users.movie_id_to_ratings, train_users.user_to_movie_id_to_rating, 
                                                         train_users.user_to_target_movie_id, train_users.user_to_target_rating, train_users.feature_1, train_users.feature_2)

#populate test data (feature 1 and feature 2)
load_feature_1_and_2(test_users.target_movies, test_users.movies_in_order, user_to_data_test, test_users.movie_id_to_ratings, test_users.user_to_movie_id_to_rating, 
               test_users.user_to_target_movie_id,
               test_users.user_to_target_rating, test_users.feature_1, test_users.feature_2)


def pre_svd(movie_id_to_average_rating, movies_in_order, user_to_ratings_full_transform, user_to_ratings_full, user_to_target_index_full, 
               user_to_movie_id_to_rating, user_to_target_movie_id):
    """
    Populate the lists user_to_ratings_full and user_to_ratings_full_transform 
    User_to_ratings_full_transform is used for svd because it includes entries from all movies in movies_in_order
    It also transforms the data in user_to_ratings_full by subtracting the movie rating mean
    This means that the transformed value at the indices of unwatched movies and index coresponding to target movies are zero
    """
    for i in range(len(user_to_movie_id_to_rating)):
        ratings = []
        transformed_ratings = []

        #the index of the target movie within the entire movies_in_order ordered set
        index = 0

        for movie_id in movies_in_order:
            if movie_id == user_to_target_movie_id[i]:
                user_to_target_index_full.append(index)
                ratings.append(movie_id_to_average_rating[movie_id])
                transformed_ratings.append(movie_id_to_average_rating[movie_id] - movie_id_to_average_rating[movie_id]) 
            elif movie_id in user_to_movie_id_to_rating[i].keys():
                ratings.append(user_to_movie_id_to_rating[i][movie_id])
                transformed_ratings.append(user_to_movie_id_to_rating[i][movie_id] - movie_id_to_average_rating[movie_id])
            else:
                ratings.append(movie_id_to_average_rating[movie_id])
                transformed_ratings.append(movie_id_to_average_rating[movie_id] - movie_id_to_average_rating[movie_id])
            index +=1
        #user_to_ratings_full is just for show
        user_to_ratings_full.append(ratings)
        #per movie averages have been subtracted (data is ready for svd)
        user_to_ratings_full_transform.append(transformed_ratings)



def svd_full(user_to_ratings_full_transform, n, movie_id_to_average_rating):
    """
    1. get the svd of the user_to_ratings_full_transform 
    2. truncate each factor to 20 components
    3. multiply the trauncated components together (U X s) X V 
    4. scale back the values to the orginal rating scale (1-5) and return result
    """
    U, s, V = np.linalg.svd(user_to_ratings_full_transform, full_matrices=False)
    
    #simplify factors to n features
    s=np.diag(s)
    s=s[0:n,0:n]
    U=U[:,0:n]
    V=V[0:n,:]

    #reconstruct to a new array
    Us = np.dot(U,s)
    UsV = np.dot(Us,V)

    x = np.tile(list(movie_id_to_average_rating.values()), (UsV.shape[0],1))

    #this tranforms the UsV row by row into the original rating scale (1-5)
    UsV = UsV + x

    #be consistent with data structures...
    return list(UsV)



def load_feature_3():
    """
    populate feature_3 with a method loosley outlined here:
    find the average ratings for movies 
    pre_svd writes a rating for every movie for every user as well as a transformed version of those rating with the averages found above
    then use the output of the svd_full function by row for user and by column for the target movie rating prediction
    """

    #Every movie ever seen by any user in either the test and train sets
    all_movies_in_order = train_users.movies_in_order|test_users.movies_in_order


    #Note: When a movie has a number of target ratings and non-target ratings, then only the non-target ratings are used...
    #to form the movies average rating

    #Note: there is a difference between non-target ratings between movie_id_to_average_rating_train and movie_id_to_average_rating_full.
    #movie_id_to_average_rating_train considers the train set and movie_id_to_average_rating_full considers the train and test set

    #Note: When a movie has only target ratings for movie_id_to_average_rating_train or movie_id_to_average_rating_full...
    #instead of using the mean of the actual target ratings, the movies average rating takes on the value of overall_average_train.
    #this is used to simlulate the potential movies to be rated for a new user that have no ratings in the existing data.

    #Note: The code below deliniates two different averages for valid movies, a train average and a train+test or full average
    #the train averge is used to train the model and the train+test averages are used to test the model
    #if the full averge is used for testing, why does the full average data have train data???
    #because it has already been processed by the system and it might as well be stored because...
    #it is critical data for svd predictions!!!

    movie_id_to_average_rating_train = dict()
    movie_id_to_average_rating_full = dict()

    for movie in all_movies_in_order:
        temp = 0
        if(movie in train_users.movie_id_to_ratings and movie in test_users.movie_id_to_ratings):
            for rating in train_users.movie_id_to_ratings[movie]:
                temp+=rating
            movie_id_to_average_rating_train[movie] = float(temp/len(train_users.movie_id_to_ratings[movie])) 

            for rating in test_users.movie_id_to_ratings[movie]:
                temp+=rating
            movie_id_to_average_rating_full[movie] = float(temp/(len(train_users.movie_id_to_ratings[movie])+len(test_users.movie_id_to_ratings[movie])))  

        elif(movie in train_users.movie_id_to_ratings):
            for rating in train_users.movie_id_to_ratings[movie]:
                temp+=rating
            movie_id_to_average_rating_train[movie] = float(temp/len(train_users.movie_id_to_ratings[movie]))
            movie_id_to_average_rating_full[movie] = movie_id_to_average_rating_train[movie]

        elif(movie in test_users.movie_id_to_ratings):        
            if(movie in train_users.target_movies):
                movie_id_to_average_rating_train[movie] = overall_average_train

            for rating in test_users.movie_id_to_ratings[movie]:
                temp+=rating
            movie_id_to_average_rating_full[movie] = float(temp/len(test_users.movie_id_to_ratings[movie]))
        else:
            if(movie in train_users.target_movies):
                movie_id_to_average_rating_train[movie] = overall_average_train
                movie_id_to_average_rating_full[movie] = overall_average_train
            else:
                movie_id_to_average_rating_full[movie] = overall_average_train
   

    #note: the three variables below are mirrored in the train_users object
    #these variables are for the full set (all_movies_in_order)

    full_user_to_ratings_full = [] #all the ratings for each user for the movies corresponding to the order of the ordered set (all_movies_in_order)
    full_user_to_ratings_full_transform = [] #all the transformed ratings for each user for the movies corresponding to the order of the ordered set (all_movies_in_order)
    full_user_to_target_index_full = [] #for each user, the index of the target movie corresponding to the order of (all_movies_in_order)


    #combining the watched movies from train and then test users
    full_user_to_movie_id_to_rating  = train_users.user_to_movie_id_to_rating + test_users.user_to_movie_id_to_rating
    #combining the id of the target movie from train and then test users
    full_user_to_target_movie_id = train_users.user_to_target_movie_id + test_users.user_to_target_movie_id


    #the two function calls below are used to populate user_to_ratings_full_transform, user_to_ratings_full, and user_to_target_index_full (both train and full versions). 
    #user_to_ratings_full_transform is scaled with the movie_id_to_average_rating (both train and full versions)

    pre_svd(movie_id_to_average_rating_train, train_users.movies_in_order, train_users.user_to_ratings_full_transform, train_users.user_to_ratings_full, 
            train_users.user_to_target_index_full, train_users.user_to_movie_id_to_rating, train_users.user_to_target_movie_id)

    pre_svd(movie_id_to_average_rating_full, all_movies_in_order, full_user_to_ratings_full_transform, full_user_to_ratings_full, full_user_to_target_index_full, 
                full_user_to_movie_id_to_rating, full_user_to_target_movie_id)


    #In practice, there is a train and a test set, the train set is what the database has on record
    #the test data will usually be data that hasn't been seen before that can include any number of test users

    #When train_users.user_to_ratings_full_transform is used as the input of the svd function, 
    #svd_out_train is used to produce predictions used to train the model

    #When full_user_to_ratings_full_transform is used as the input of the svd function,
    #svd_out_full is used to produce predictions used to test the model


    #n = 20 seemed to be close to the highest performing constant for the above configuration
    svd_out_train = svd_full(train_users.user_to_ratings_full_transform, 20, movie_id_to_average_rating_train)
    svd_out_full = svd_full(full_user_to_ratings_full_transform, 20, movie_id_to_average_rating_full)

    #here the smaller svd provides predictions used to train the model
    for i in range(len(train_users.user_to_ratings_full_transform)):
        train_users.feature_3.append(svd_out_train[i][train_users.user_to_target_index_full[i]])

    #here the larger svd provides predictions used to test the model
    for i in range(len(full_user_to_ratings_full_transform) - len(train_users.user_to_ratings_full_transform)):
        test_users.feature_3.append(svd_out_full[i+len(train_users.user_to_ratings_full_transform)][full_user_to_target_index_full[i+len(train_users.user_to_ratings_full_transform)]])

load_feature_3()


#this is just used to show how the features approximate the target rating
print("Feature_1 to target comparison (train):")
print(train_users.feature_1[0:5])
print(train_users.user_to_target_rating[0:5])

print("Feature_1 to target comparison (test):")
print(test_users.feature_1[0:5])
print(test_users.user_to_target_rating[0:5])

print("Feature_2 to target comparison (train):")
print(train_users.feature_2[0:5])
print(train_users.user_to_target_rating[0:5])

print("Feature_2 to target comparison (test):")
print(test_users.feature_2[0:5])
print(test_users.user_to_target_rating[0:5])

print("Feature_3 to target comparison (train):")
print(train_users.feature_3[0:5])
print(train_users.user_to_target_rating[0:5])

print("Feature_3 to target comparison (test):")
print(test_users.feature_3[0:5])
print(test_users.user_to_target_rating[0:5])


#just for code clarity
del user_to_data_train
del user_to_data_test


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jackson\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Feature_1 to target comparison (train):
[3.411290322580645, 3.896551724137931, 3.1557377049180326, 3.347457627118644, 3.588235294117647]
[4.0, 4.0, 4.0, 4.0, 3.0]
Feature_1 to target comparison (test):
[2.111111111111111, 3.5714285714285716, 3.5625, 3.0, 4.277777777777778]
[4.0, 3.0, 0.5, 4.0, 3.0]
Feature_2 to target comparison (train):
[3.50229426239177, 3.8492242609426492, 3.289531210338809, 3.2428965419785527, 3.5535379743743403]
[4.0, 4.0, 4.0, 4.0, 3.0]
Feature_2 to target comparison (test):
[1.4905347168091443, 3.9911113691973883, 3.5795614530307445, 3.0, 3.8349874609669294]
[4.0, 3.0, 0.5, 4.0, 3.0]
Feature_3 to target comparison (train):
[3.5608110219962956, 4.40352458988661, 4.068151290100832, 3.9148678390564107, 3.646058396510628]
[4.0, 4.0, 4.0, 4.0, 3.0]
Feature_3 to target comparison (test):
[3.69694126783935, 3.4578857864149306, 3.761380804191325, 3.4686353917702677, 3.55634251892873]
[4.0, 3.0, 0.5, 4.0, 3.0]


In [13]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


def test_parameters(nof_runs, layers, train_input_features, test_input_features):
    """Test_parameters for a number of runs and return performance results"""
    train_inputs = [list(pair) for pair in train_input_features]
    test_inputs = [list(pair) for pair in test_input_features]
    return average_results(nof_runs, layers, train_inputs, test_inputs)
    

def average_results(nof_runs, layers, train_inputs, text_inputs):
    """Average the performance results for a number of models with identical inputs"""
    no_rounding = 0
    rounding = 0
    for _ in range(nof_runs):
        pair = train_and_test(layers, train_inputs, text_inputs)
        no_rounding+=pair[0]
        rounding+=pair[1]
    return float(no_rounding/nof_runs), float(rounding/nof_runs)


def train_and_test(layers, train_inputs, test_inputs):
    """Build, train, and test a model, then return accuracy scores"""

    # nn model (worse performance)
    # reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)

    # linear regression (better performance)
    reg = LinearRegression()

    #train model
    reg.fit(train_inputs, train_users.user_to_target_rating)

    #print importance of the different input features to the model
    results = permutation_importance(reg, train_inputs, train_users.user_to_target_rating)
    print("Feature Importances: ", results["importances_mean"])

    #make predictions for test inputs
    predictions = reg.predict(test_inputs)

    #test with and without roundings...
    #note: the actual ratings a user makes must be divisable by .5 
    rounded_predictions = []
    for item in predictions:
        rounded_predictions.append(float(round(item*2)/2.0))

    #evaluation metric 1:
    return(r2_score(test_users.user_to_target_rating, predictions), 
        r2_score(test_users.user_to_target_rating, rounded_predictions))

    #evaluation metric 2:
    # return(mean_squared_error(test_users.user_to_target_rating, predictions), 
    #         mean_squared_error(test_users.user_to_target_rating, rounded_predictions))



# the current test is the average scores (currently r2_score) for 100 models trained on the same inputs
# the hidden layers are (10,10,10) but the linear model is the current model being used which does not user layers
# and the best combination of inputs features (feature_1 and feature_3) are used

avg_scores = test_parameters(100, (10,10,10), 
    zip(train_users.feature_1, train_users.feature_3),
      zip(test_users.feature_2, test_users.feature_3))


print("Average r2_score without rounding: ",avg_scores[0])
print("Average r2_score with rounded prediction to nearest .5 (note: actual users ratings must be divisibl by .5): ",avg_scores[1])


# results:
# linear regression, train users with 50-75 ratings, test users with 5-10 ratings, using feature_1 and feature_3
# Average r2_score without rounding:  0.1420517022263964
# Average r2_score with rounded prediction to nearest .5 (note: actual users ratings must be divisibl by .5):  0.12610854913089753

# mlp model, train users with 50-75 ratings, test users with 5-10 ratings, using feature_1 and feature_3
# Average r2_score without rounding:  0.1237852059540125
# Average r2_score with rounded prediction to nearest .5 (note: actual users ratings must be divisibl by .5):  0.1061582121319617

Feature Importances:  [0.16150779 0.33165977]
Feature Importances:  [0.15674213 0.34454896]
Feature Importances:  [0.1704534  0.32761626]
Feature Importances:  [0.16937697 0.34132943]
Feature Importances:  [0.1593629  0.31896023]
Feature Importances:  [0.15534316 0.32904467]
Feature Importances:  [0.17155619 0.32842754]
Feature Importances:  [0.16613902 0.32811516]
Feature Importances:  [0.16712634 0.33305428]
Feature Importances:  [0.16013826 0.32319303]
Feature Importances:  [0.1546703  0.33343778]
Feature Importances:  [0.16302135 0.33790305]
Feature Importances:  [0.15366697 0.3278223 ]
Feature Importances:  [0.17151332 0.34042882]
Feature Importances:  [0.15239914 0.32839484]
Feature Importances:  [0.16029656 0.33325841]
Feature Importances:  [0.16051491 0.34280163]
Feature Importances:  [0.1586315  0.33745531]
Feature Importances:  [0.16774006 0.34151805]
Feature Importances:  [0.1718792  0.34536468]
Feature Importances:  [0.1695496  0.33393791]
Feature Importances:  [0.1584937  